# Method

In [ ]:
!pip install activeSVC

In [ ]:
import numpy as np
import time
import random
import os

from sklearn.preprocessing import normalize 
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
from matplotlib import gridspec

from activeSVC import min_complexity_h5py, min_acquisition_h5py
import pandas as pd
import pickle
import os, psutil
import resource

from scipy import io as sio
import seaborn as sns
import h5py
import pandas as pd

def text_create(path, name, msg):
    full_path = path + "/" + name + '.pickle'
    f=open(full_path,'wb') 
    pickle.dump(msg,f)
    f.close()

class TimerError(Exception):
     """A custom exception used to report errors in use of Timer class"""

class Timer:
    def __init__(self):
        self._start_time = None

    def start(self):
        if self._start_time is not None:
            raise TimerError(f"Timer is running. Use .stop() to stop it")

        self._start_time = time.perf_counter()

    def stop(self):
        if self._start_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")

        elapsed_time = time.perf_counter() - self._start_time
        self._start_time = None
        print(f"Total run time: {elapsed_time:0.4f} seconds")
        return elapsed_time





# Load Data

## Please download the data folder from this dropbox link, as the dataset is too large to upload or auto-downlaod here.

https://www.dropbox.com/sh/8b3kf59chod6o5j/AADiLQbHZFQqN4FXGda_N5MZa?dl=0

In [ ]:
df = pd.read_csv('./data/clusters10.csv', index_col=0)
y=df['Cluster'].values
classes = np.unique(y)
keys=classes

f = h5py.File('./data/1M_neurons_filtered_gene_bc_matrices_h5.h5', 'r')
h5data=f['mm10']
gene_names=h5data['gene_names']
shape=h5data['shape']

f = h5py.File('./data/norm_1M_neurons_filtered_gene_bc_matrices_h5.h5', 'r')
h5data=f['mm10']
data_cell=h5data['data']
indices_cell=h5data['indices']
indptr_cell = h5data['indptr']

f = h5py.File('./data/norm_transpose_1M_neurons_filtered_gene_bc_matrices_h5.h5', 'r')
h5data=f['mm10']
data_gene=h5data['data']
indices_gene=h5data['indices']
indptr_gene = h5data['indptr']


genelist=np.array(gene_names)
gene=[]
for i in range(genelist.shape[0]):
    gene.append(genelist[i].decode())
gene=np.array(gene)

# Select Genes and Save Results

In [ ]:
'''
min-complexity random
Sometimes it shows ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0
This is because of the settings of algorithm. 
Please set a larger num_samples and init_samples and then re-run this cell. 
Or set parameter class_weight='balanced' and balance=True and then re-run the cell. 
'''

idx = np.arange(shape[1])
random.shuffle(idx)
idx_train=idx[:int(shape[1]*4/5)]
idx_test=idx[int(shape[1]*4/5):]
        
num_features = 50
num_samples=20
init_samples=20
balance=True

path='results/50genes_20cells'

try:
    os.mkdir('results')
except OSError:
    print ("Creation of the directory %s failed" % 'results')
else:
    print ("Successfully created the directory %s " % 'results')
try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)
    
    
if __name__ == '__main__':

    t=Timer()
    t.start()
    feature_selected, num_samples_list, train_errors,test_errors,train_scores,test_scores,step_times= min_complexity_h5py(
        data_cell,indices_cell,indptr_cell,data_gene,indices_gene,indptr_gene, y, shape,idx_train,idx_test, 
        num_features=num_features,num_samples=num_samples,init_features=1,init_samples=init_samples, balance=balance,class_weight='balanced')
    elapsed_time=t.stop()
    
    memorys=[]
    memorys.append(psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2)
    memorys.append(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/ 1024 ** 2)
    
    text_create(path,'feature_selected',feature_selected)
    text_create(path,'error',train_errors+test_errors)
    text_create(path,'accuracy',train_scores+test_scores)
    text_create(path,'num_samples_list',num_samples_list)
    text_create(path,'genes_name',gene[feature_selected])
    text_create(path,'elapsed_time',elapsed_time)
    text_create(path,'memory',memorys)

    plt.figure(figsize=(8,8))
    plt.plot(train_scores,linewidth=2)
    plt.plot(test_scores,linewidth=2)
    plt.legend(['train acc','test acc'],prop = {'size':18})
    plt.xlabel('number of genes',fontdict={'weight':'normal','size': 18})
    plt.ylabel('accuracy',fontdict={'weight':'normal','size': 18})
    plt.tick_params(labelsize=18)
    plt.savefig(path+'/acc.pdf', bbox_inches="tight")

feature1=np.copy(feature_selected)

In [ ]:
'''
min-cell
Sometimes it shows ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0
This is because of the settings of algorithm. 
Please set a larger num_samples and init_samples and then re-run this cell. 
Or set parameter class_weight='balanced' and balance=True and then re-run the cell. 
'''

idx = np.arange(shape[1])
random.shuffle(idx)
idx_train=idx[:int(shape[1]*4/5)]
idx_test=idx[int(shape[1]*4/5):]
        
num_features = 50
num_samples=100
init_samples=100
balance=True

path='results/50genes_100cells'

try:
    os.mkdir('results')
except OSError:
    print ("Creation of the directory %s failed" % 'results')
else:
    print ("Successfully created the directory %s " % 'results')
try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)
    
    
if __name__ == '__main__':

    t=Timer()
    t.start()
    feature_selected, num_samples_list, samples_global, train_errors,test_errors,train_scores,test_scores,step_times= min_acquisition_h5py(
        data_cell,indices_cell,indptr_cell,data_gene,indices_gene,indptr_gene, y, shape,idx_train,idx_test,
        num_features=num_features,num_samples=num_samples,init_features=1,init_samples=init_samples,balance=balance, class_weight='balanced')
    elapsed_time=t.stop()
    
    memorys=[]
    memorys.append(psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2)
    memorys.append(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/ 1024 ** 2)
    
    text_create(path,'feature_selected',feature_selected)
    text_create(path,'error',train_errors+test_errors)
    text_create(path,'accuracy',train_scores+test_scores)
    text_create(path,'num_samples_list',num_samples_list)
    text_create(path,'genes_name',gene[feature_selected])
    text_create(path,'elapsed_time',elapsed_time)
    text_create(path,'memory',memorys)

    plt.figure(figsize=(8,8))
    plt.plot(train_scores,linewidth=2)
    plt.plot(test_scores,linewidth=2)
    plt.legend(['train acc','test acc'],prop = {'size':18})
    plt.xlabel('number of genes',fontdict={'weight':'normal','size': 18})
    plt.ylabel('accuracy',fontdict={'weight':'normal','size': 18})
    plt.tick_params(labelsize=18)
    plt.savefig(path+'/acc.pdf', bbox_inches="tight")
    
    plt.figure(figsize=(8,5))
    plt.plot(num_samples_list,linewidth=5)
    plt.xlabel('number of genes',fontdict={'weight':'normal','size': 18})
    plt.ylabel('number of cells acquired',fontdict={'weight':'normal','size': 18})
    plt.tick_params(labelsize=18)
    plt.savefig(path+'/cells.pdf', bbox_inches="tight")

feature2=np.copy(feature_selected)

# Plots

In [ ]:
with open('./data/data_plot.pickle','rb') as handle:
    matrix=pickle.load(handle)
    target=pickle.load(handle)

alpha = 10000
data_show = np.log1p(matrix*alpha)

In [ ]:
plt.figure(figsize=(9,6))
c_cmap = plt.get_cmap('tab10', len(classes))
plt.scatter(tsne1[sampling[:30000]],tsne2[sampling[:30000]],c=target,s=1,cmap=c_cmap)
cbar=plt.colorbar(ticks=np.arange(1,len(classes)+1))
plt.xticks([])
plt.yticks([])
cbar.ax.set_yticklabels(keys,fontdict={'weight':'normal','size': 18})
plt.savefig(path+'/tsne.pdf',bbox_inches="tight")

In [ ]:
feature_compare=[8435,8952,1815,10593,14065,19901,14025,23521,16067,18983,22327,2340]
num_col=12
size=len(feature_compare)
plt.figure(figsize=(3*num_col,2.5*(int(size/num_col)+1)))
gs = gridspec.GridSpec(int(size/num_col)+1, num_col, width_ratios=[7]*num_col)

for i in range(size):
    f=feature_compare[i]
    ax0 = plt.subplot(gs[i])
    ax0.set_title(gene_name[f],fontdict={'weight':'normal','size': 18})
    plt.scatter(tsne1[sampling[:30000]],tsne2[sampling[:30000]],c=data_show[:,f],s=0.01)
    ax0.set_xticks([])
    ax0.set_yticks([])
plt.savefig(path+'/compare_genes.png',dpi=300,  facecolor='white',bbox_inches="tight")   

In [ ]:
feature_selected=list(set(feature1)|set(feature2)|set(feature3))

In [ ]:
f_first=[8435,8952,1815,20478,1368,18521,17464,14438,19595,2838,17491,24554]
features_show=f_first+list(set(feature_selected)-set(f_first))
    
num_col=12

size=len(features_show)
plt.figure(figsize=(3*num_col,2.5*(int(size/num_col)+1)))
gs = gridspec.GridSpec(int(size/num_col)+1, num_col, width_ratios=[7]*num_col)

for i in range(size):
    f=features_show[i]
    ax0 = plt.subplot(gs[i])
    ax0.set_title(gene_name[f],fontdict={'weight':'normal','size': 18})
    plt.scatter(tsne1[sampling[:30000]],tsne2[sampling[:30000]],c=data_show[:,f],s=0.01)
    ax0.set_xticks([])
    ax0.set_yticks([])
plt.savefig(path+'/genes.png',dpi=300,  facecolor='white',bbox_inches="tight")   



In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
coef_matrix=np.zeros((len(feature_compare),len(feature_selected)))
for i in range(len(feature_compare)):
    for j in range(len(feature_selected)):
        coef_matrix[i,j]=np.corrcoef(matrix[:,feature_compare[i]],matrix[:,feature_selected[j]])[0,1]
np.savetxt(path+"/coef_matrix.csv", coef_matrix, delimiter=",")
plt.figure(figsize=(15,3))
sns.clustermap(coef_matrix,figsize=(30,8.5),cmap="viridis",method='average',
               xticklabels=gene_name[feature_selected].tolist(),yticklabels=gene_name[feature_compare].tolist())
plt.savefig(path+'/coef_matrix.png',dpi=300,  facecolor='white',bbox_inches="tight")
